# Session 7 — Interactive EDA with Plotly

Objectives
- Build interactive, exploratory plots with Plotly (plotly.express and plotly.graph_objects).
- Use Plotly FigureWidget for interactive notebook exploration.



In [1]:
# Run in Colab
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

sns.set_style("whitegrid")
tips = sns.load_dataset("tips")
tips.head()


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
# 1) Interactive histogram + KDE-like overlay (density via histogram_norm)
fig = px.histogram(tips, x="total_bill", nbins=30, histnorm="probability density",
                   marginal="rug", title="Total bill distribution (density)")
fig.update_traces(marker_color="#1f77b4", opacity=0.75)
fig.show()


In [5]:
# 2) Interactive scatter with regression line (Plotly doesn't add regression by default; compute fit)
x = tips["total_bill"]
y = tips["tip"]
coef = np.polyfit(x, y, 1)
fit_y = np.poly1d(coef)(x)
fig = px.scatter(tips, x="total_bill", y="tip", color="day", hover_data=["size"],
                 title="Tip vs Total bill (interactive)")
fig.add_traces(go.Scatter(x=x, y=fit_y, mode="lines", line=dict(color="black", width=2),
                          name="Linear fit", hoverinfo="skip"))
fig.show()


In [6]:
# 3) Box + jitter (violin-like interactivity): use facet or combined subplot
fig = make_subplots(rows=1, cols=2, subplot_titles=("Boxplot by day", "Strip (jitter) by day"))
# Box
for i, d in enumerate(sorted(tips["day"].unique())):
    fig.add_trace(go.Box(y=tips.loc[tips.day==d, "tip"], name=d, boxmean="sd"), row=1, col=1)
# Jitter (strip)
fig.add_trace(go.Scatter(x=tips["day"], y=tips["tip"], mode="markers",
                         marker=dict(opacity=0.6, size=6, color="rgba(31,119,180,0.6)"),
                         name="points"), row=1, col=2)
fig.update_layout(height=450, showlegend=False, title_text="Box and points: tip by day")
fig.show()


In [7]:
# 4) Interactive correlation heatmap (hover shows values)
num_cols = ["total_bill","tip","size"]
corr = tips[num_cols].corr()
fig = px.imshow(corr, text_auto=".2f", color_continuous_scale="RdBu", origin="lower",
                title="Correlation matrix (interactive)")
fig.update_layout(height=450)
fig.show()


In [8]:
from google.colab import output
output.enable_custom_widget_manager()

In [12]:
day_of_week = ['Thur', 'Fri', 'Sat', 'Sun']

In [14]:
# 5) Small interactive dashboard inside notebook using widgets (FigureWidget)
from ipywidgets import HBox, VBox, Dropdown, IntSlider, Output
from plotly.graph_objs import FigureWidget

fw = FigureWidget(px.scatter(tips, x="total_bill", y="tip", color="day"))
out = Output()

day_dropdown = Dropdown(options=["All"] + day_of_week, description="Day:")
size_slider = IntSlider(value=1, min=1, max=int(tips["size"].max()), description="Min size:")

def update_plot(change):
    sel_day = day_dropdown.value
    min_size = size_slider.value
    df = tips.copy()
    if sel_day != "All":
        df = df[df["day"] == sel_day]
    df = df[df["size"] >= min_size]
    with fw.batch_update():
        fw.data = []  # clear
        # re-add traces per day
        for d in sorted(df["day"].unique()):
            sub = df[df["day"]==d]
            fw.add_scatter(x=sub["total_bill"], y=sub["tip"], mode="markers", name=d)
    out.clear_output()
    with out:
        print(f"Showing {len(df)} rows — Day: {sel_day}, Min size: {min_size}")

day_dropdown.observe(update_plot, names="value")
size_slider.observe(update_plot, names="value")

display(VBox([HBox([day_dropdown, size_slider]), fw, out]))


In [21]:
tips[(tips['day'] == 'Thur') & (tips['size'] >= 3)]

,total_bill,tip,sex,smoker,day,time,size
77,27.20,4.00,Male,No,Thur,Lunch,4
85,34.83,5.17,Female,No,Thur,Lunch,4
119,24.08,2.92,Female,No,Thur,Lunch,4
125,29.80,4.20,Female,No,Thur,Lunch,6
129,22.82,2.18,Male,No,Thur,Lunch,3
141,34.30,6.70,Male,No,Thur,Lunch,6
142,41.19,5.00,Male,No,Thur,Lunch,5
143,27.05,5.00,Female,No,Thur,Lunch,6
146,18.64,1.36,Female,No,Thur,Lunch,3
197,43.11,5.00,Female,Yes,Thur,Lunch,4
